In [3]:
import pandas as pd
import pymrmr

In [4]:
df = pd.read_csv('GSE84465_GBM_All_data.csv', delimiter=' ')

# make samples as rows
df = df.T

# remove the last 5 columns 
df = df.iloc[: , :-5]
df

,1/2-SBSRNA4,A1BG,A1BG-AS1,A1CF,A2LD1,A2M,A2ML1,A2MP1,A4GALT,A4GNT,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,tAKR
1001000173.G8,1,0,0,0,0,0,0,0,0,0,...,0,30,0,0,0,0,0,0,0,0
1001000173.D4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1001000173.B4,0,0,0,0,0,0,0,0,0,0,...,0,0,40,33,0,0,834,0,0,0
1001000173.A2,0,0,1,0,0,0,34,0,0,0,...,0,0,41,61,0,0,0,33,1,0
1001000173.E2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,439,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001000271.F1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1001000271.D4,0,61,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1001000271.C1,0,0,0,0,0,61,0,0,2,0,...,0,1,98,19,0,0,18,0,64,0
1001000271.H7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# remove columns and rows that have sum = 0 -> no information/expression
df = df.loc[(df.sum(axis=1) != 0), (df.sum(axis=0) != 0)]

In [8]:
df_sum_col = df.sum()
df_sum_col
# df_sum_col.mean()

1001000173.G8     971747
1001000173.D4     509929
1001000173.B4    1101191
1001000173.A2    1310084
1001000173.E2    1218726
                  ...   
1001000271.F1     705806
1001000271.D4     559750
1001000271.C1    1094316
1001000271.H7       5180
1001000271.H2    1088850
Length: 3589, dtype: int64

In [6]:
# MIQ -> Mutual Information Quotient
# MID -> Mutual Information Difference
pymrmr.mRMR(df, 'MIQ', 100) # dataframe, feature selection scheme, # of features